<h1> Load annotations </h1>

In [ ]:
import os
import sys
import random
import math
import pickle
import re
import time
import numpy as np
import cv2
# import matplotlib
# import matplotlib.pyplot as plt
import keras
import skimage.io

LIB_DIRECTORY = '/root/alok/repos/cv_research/lib/'
sys.path.insert(0, os.path.join(LIB_DIRECTORY, 'maskrcnn'))
from mrcnn.config import Config
import mrcnn.utils as utils
import mrcnn.model as modellib
import mrcnn.visualize as visualize
from mrcnn.model import log
import mcoco.coco as coco
# import mextra.utils as extra_utils
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

import json
import os
import glob
import sys
LIB_DIRECTORY = '/root/alok/repos/cv_research/lib/'
sys.path.insert(0, LIB_DIRECTORY)

import pandas as pd
from pycocotools.coco import COCO

%matplotlib inline
%config IPCompleter.greedy=True
BASE_DIR = '/root/data/models/erko/mask_rcnn_instance_segmentation'
DATA_DIR = '/root/data/erko/'
WEIGHTS_DIR = os.path.join(BASE_DIR, "weights")
MODEL_DIR = os.path.join(BASE_DIR, "logs", "body_part_segmentation_20181031_21H02")



In [ ]:
annotation_file = '/root/alok/data/images/annotation_file_test_set.json'

In [ ]:
coco_output = json.load(open(annotation_file))

<h1> Visualize the Results </h1>

In [ ]:
from matplotlib import pyplot as plt
from matplotlib.patches import Rectangle
from matplotlib.collections import PatchCollection

In [ ]:
coco = COCO(annotation_file)

In [ ]:
def get_ax(rows=1, cols=1, size=8):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    
    Change the default size attribute to control the size
    of rendered images
    """
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax

In [ ]:
def plot_annotations(image_id):    
    image_data = coco.loadImgs([image_id])[0]
    image_file_path = image_data['local_path']
    annotation_ids = coco.getAnnIds(imgIds=[image_id])
    annotations = coco.loadAnns(annotation_ids)

    # load and display instance annotations
    image = skimage.io.imread(image_file_path)
    f, ax = plt.subplots(1, figsize=(20, 20))
    ax.imshow(image)
    ax.axis('off')
    coco.showAnns(annotations)
    
    # display bounding boxes
    for ann in annotations:
        bbox = ann['bbox']
        rectangle = Rectangle((bbox[1], bbox[0]), bbox[3]-bbox[1], bbox[2]-bbox[0], edgecolor='w', facecolor=None, fill=False, linestyle='--', linewidth=2)
        ax.add_patch(rectangle)
#         category_id = ann['category_id']
        category_id = ann['id']
        ax.text(bbox[1], bbox[0] - 10, category_id, fontsize=16, color='w')


    
    


In [ ]:
plot_annotations(527)

In [ ]:
plot_annotations(2222)

<h1> Determine left-right matches (bipartite graph approach) </h1>

In [ ]:
image_ids = coco.getImgIds()
images = coco.loadImgs(image_ids)
stereo_frame_pairs = {}
for image in images:
    local_path = image['local_path']
    sfp_id = int(local_path.split('/')[-3])
    side = 'left' if 'left' in local_path else 'right'
    if sfp_id not in list(stereo_frame_pairs.keys()):
        stereo_frame_pairs[sfp_id] = {}
    stereo_frame_pairs[sfp_id][side] = image['id']
    
stereo_frame_pairs = list(stereo_frame_pairs.values())
    
    

In [ ]:
def get_fish_detections(image_id, full_fish_category_id=1):
    
    # get all annotations corresponding to fish detections
    
    fish_detections = []
    annotation_ids = coco.getAnnIds(imgId=[image_id])
    annotations = coco.loadAnns(annotation_ids)
    full_fish_annotations = [ann for ann in annotations if ann['category_id'] == full_fish_category_id]
    for full_fish_annotation in full_fish_annotations:
        fish_detection = {
            'full_fish_annotation': full_fish_annotation, 
            'body_part_annotations': []
        }
        fish_detection['bounding_box'] = transform_coco_bbox(full_fish_annotation['bbox'])
        fish_detections.append(fish_detection)
        
    # for each fish detection, append
    body_part_annotations = [ann for ann in annotations if ann['category_id'] != FULL_FISH_CATEGORY_ID]
    for body_part_annotation in body_part_annotations:
        body_part_centroid = get_centroid_from_coco_bbox(body_part_annotation['bbox'])
        body_part_matched_to_fish_detection = False
        for fish_detection in fish_detections:
            body_part_inside_detection = \
                determine_if_body_part_falls_inside_detection(body_part_centroid, fish_detection['bounding_box'])
            if body_part_inside_detection:
                fish_detection['body_part_annotations'].append(body_part_annotation)
                body_part_matched_to_fish_detection = True
        if not body_part_matched_to_fish_detection:
            unmatched_body_parts.append(body_part_annotation)
    fish_detections.extend(fish_detections_in_image)
    
    
    
    

In [ ]:
def left_right_matching(left_fish_detections, right_fish_detections):

    # let's use x1, x2 to match bboxes
    left_corners = []
    for ann in left_fish_detections:
        bbox = ann['bbox']
        corner = [bbox[2], bbox[0]]
        left_corners.append(corner)

    right_corners = []
    for ann in right_fish_detections:
        bbox = ann['bbox']
        # centroid = [(bbox[3] - bbox[1])/2.0, (bbox[2] - bbox[0])/2.0]
        corner = [bbox[2], bbox[0]]
        right_corners.append(corner)

    # euclidean distance in (x1, x2) space
    cost_matrix = euclidean_distances(left_corners, right_corners)
    
    # hungarian algorithm to minimize weights in bipartite graph
    row_ind, col_ind = linear_sum_assignment(cost_matrix)
    matched_fish_detections = []
    for r, c in zip(row_ind, col_ind):
        matched_fish_detections.append((left_fish_detections[r], right_fish_detections[c]))
    
    return matched_fish_detections


In [ ]:
def match_body_parts(annotations, categories, whole_fish="salmon"):
    """ take the predictions on a single frame and associate body parts with whole salmon using a unique identifier"""
    whole_fish_id = [cat for cat in categories if cat['name'] == whole_fish][0]['id']
    fish = []
    body_parts = []
    fish_counter = 0

    # first we separate whole salmon from body parts and give a unique identifier
    for annotation in annotations:
        if annotation['category_id'] == whole_fish_id:
            annotation['fish_id'] = fish_counter
            fish.append(annotation)
            fish_counter += 1
        else:
            body_parts.append(annotation)

    # second we match body parts with whole salmon
    # @TODO (Thomas) this is not great because it does not take edges cases into account.
    for part in body_parts:
        bbox = part['bbox']
        part_centroid = [np.mean([bbox[0], bbox[2]]), np.mean([bbox[1], bbox[3]])]
        for f in fish:
            bbox = f['bbox']
            if bbox[0] < part_centroid[0] < bbox[2] and bbox[1] < part_centroid[1] < bbox[3]:
                part['fish_id'] = f['fish_id']
                break

    return annotations


In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from PIL import Image, ImageDraw
from scipy.optimize import linear_sum_assignment
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import euclidean_distances


FOCAL_LENGTH = 0.0107
BASELINE = 0.135
PIXEL_SIZE_M = 3.45 * 1e-6
FOCAL_LENGTH_PIXEL = FOCAL_LENGTH / PIXEL_SIZE_M
IMAGE_SENSOR_WIDTH = 0.01412
IMAGE_SENSOR_HEIGHT = 0.01412
RANDOM_STATE = 170
DENSITY = 1e6  # g per cubic meter

LENGTH_FACTOR = 44.0  # where is this coming from? DATA
COST_THRESHOLD = 100.0  # another magic number

MODEL_PATH = '/root/data/models/biomass/model.pickle'
COMPONENTS_PATH = '/root/data/models/biomass/components.npy'
IQRS_PATH = '/root/data/models/biomass/iqrs.pkl'

CATEGORIES = [
    {'id': 1, 'name': 'salmon'},
    {'id': 2, 'name': 'Head'},
    {'id': 3, 'name': 'Caudal Fin'},
    {'id': 4, 'name': 'Dorsal Fin'},
    {'id': 5, 'name': 'Adipose Fin'},
    {'id': 6, 'name': 'Anal Fin'},
    {'id': 7, 'name': 'Pelvic Fin'},
    {'id': 8, 'name': 'Pectoral Fin'},
    {'id': 9, 'name': 'Eye'}
]

MINIMAL_BODY_PART_LIST = [2, 4, 6]


def convert_to_world_point(x, y, d):
    """ from pixel coordinates to world coordinates """
    # TODO (@Thomas) this is hard coded and this bad....
    image_center_x = 3000 / 2.0  # depth_map.shape[1] / 2.0
    image_center_y = 4096 / 2.0  # depth_map.shape[0] / 2.0
    px_x = x - image_center_x
    px_z = image_center_y - y

    sensor_x = px_x * (IMAGE_SENSOR_WIDTH / 3000)
    sensor_z = px_z * (IMAGE_SENSOR_HEIGHT / 4096)

    # d = depth_map[y, x]
    world_y = d
    world_x = (world_y * sensor_x) / FOCAL_LENGTH
    world_z = (world_y * sensor_z) / FOCAL_LENGTH
    return np.array([world_x, world_y, world_z])


def depth_from_disp(disp):
    depth = FOCAL_LENGTH_PIXEL*BASELINE / np.array(disp)
    return depth


def match_body_parts(annotations, categories, whole_fish="salmon"):
    """ take the predictions on a single frame and associate body parts with whole salmon using a unique identifier"""
    whole_fish_id = [cat for cat in categories if cat['name'] == whole_fish][0]['id']
    fish = []
    body_parts = []
    fish_counter = 0

    # first we separate whole salmon from body parts and give a unique identifier
    for annotation in annotations:
        if annotation['category_id'] == whole_fish_id:
            annotation['fish_id'] = fish_counter
            fish.append(annotation)
            fish_counter += 1
        else:
            body_parts.append(annotation)

    # second we match body parts with whole salmon
    # @TODO (Thomas) this is not great because it does not take edges cases into account.
    for part in body_parts:
        bbox = part['bbox']
        part_centroid = [np.mean([bbox[0], bbox[2]]), np.mean([bbox[1], bbox[3]])]
        for f in fish:
            bbox = f['bbox']
            if bbox[0] < part_centroid[0] < bbox[2] and bbox[1] < part_centroid[1] < bbox[3]:
                part['fish_id'] = f['fish_id']
                break

    matched_body_pars = [ann for ann in annotations if 'fish_id' in ann.keys()]
    return matched_body_pars


def left_right_matching(left_annotations, right_annotations, categories, whole_fish="salmon"):
    """match the bboxes. Return a list of matched bboxes"""
    # get the salmon category id
    whole_fish_id = [cat for cat in categories if cat['name'] == whole_fish][0]['id']

    # let's use x1, x2 to match bboxes
    left_centroids = []
    left_ids = []
    for ann in left_annotations:
        if ann['category_id'] != whole_fish_id:
            continue
        bbox = ann['bbox']
        # centroid = [(bbox[3] - bbox[1])/2.0, (bbox[2] - bbox[0])/2.0]
        corner = [bbox[2], bbox[0]]
        left_ids.append(ann['fish_id'])
        left_centroids.append(corner)

    right_centroids = []
    right_ids = []
    for ann in right_annotations:
        if ann['category_id'] != whole_fish_id:
            continue
        bbox = ann['bbox']
        # centroid = [(bbox[3] - bbox[1])/2.0, (bbox[2] - bbox[0])/2.0]
        corner = [bbox[2], bbox[0]]
        right_ids.append(ann['fish_id'])
        right_centroids.append(corner)

    # euclidean distance in (x1, x2) space
    cost_matrix = euclidean_distances(left_centroids, right_centroids)
    
    # hungarian algorithm to minimize weights in bipartite graph
    row_ind, col_ind = linear_sum_assignment(cost_matrix)

    matched_annotations = []
    for (r, c) in zip(row_ind, col_ind):
        if cost_matrix[r, c] < COST_THRESHOLD:
            left_matched_parts = [l for l in left_annotations if l['fish_id'] == left_ids[r] and l['category_id'] != whole_fish_id]
            right_matched_parts = [r for r in right_annotations if r['fish_id'] == right_ids[c] and r['category_id'] != whole_fish_id]
            matched_annotations.append([left_matched_parts, right_matched_parts])

    return matched_annotations

def weight_estimator(left_matched_parts, right_matched_parts, categories, model, components, iqrs):
    left_category_ids = [ann['category_id'] for ann in left_matched_parts]
    right_category_ids = [ann['category_id'] for ann in right_matched_parts]
    
    left_valid = all([body_part in left_category_ids for body_part in MINIMAL_BODY_PART_LIST])
    right_valid = all([body_part in right_category_ids for body_part in MINIMAL_BODY_PART_LIST])
    
    if left_valid and right_valid:
        """take left and right body parts and calculates the weights"""
        # first check that all the parts are here
        left_parts_ids = list(set([ann['category_id'] for ann in left_matched_parts]))
        right_parts_ids = list(set([ann['category_id'] for ann in right_matched_parts]))
        
        # create observation
        category_ids = [str(cat['id']) for cat in categories if cat['name'] != 'salmon']
        distances = [c + k for (i, c) in enumerate(category_ids) for k in category_ids[i + 1:]]
        dataset = {}
        for d in distances:
            dataset[d] = []

        # calculate disparities & world coordinates
        world_coordinates = {}
        for cat in categories:
            if cat['name'] == 'salmon' or cat['id'] not in MINIMAL_BODY_PART_LIST:
                continue
            # calculate left centroid
            
            left_part = [part for part in left_matched_parts if part['category_id'] == cat['id']][0]
            seg = left_part['segmentation'][0]
            poly = np.array(seg).reshape((int(len(seg) / 2), 2))
            p = [(r[0], r[1]) for r in poly]
            left_mask = Image.new('L', (4096, 3000), 0)
            ImageDraw.Draw(left_mask).polygon(p, outline=1, fill=1)
            left_mask = np.array(left_mask)
            x, y = np.nonzero(left_mask)
            left_centroid = [np.mean(x), np.mean(y)]

            # calculate right centroid
            right_part = [part for part in right_matched_parts if part['category_id'] == cat['id']][0]
            seg = right_part['segmentation'][0]
            poly = np.array(seg).reshape((int(len(seg) / 2), 2))
            p = [(r[0], r[1]) for r in poly]
            right_mask = Image.new('L', (4096, 3000), 0)
            ImageDraw.Draw(right_mask).polygon(p, outline=1, fill=1)
            right_mask = np.array(right_mask)
            x, y = np.nonzero(right_mask)
            right_centroid = [np.mean(x), np.mean(y)]

            # calculate disparity
            disparities = np.abs(left_centroid[1] - right_centroid[1])
            depth = depth_from_disp(disparities)
            world_coordinates[str(cat['id'])] = convert_to_world_point(left_centroid[0], left_centroid[1], depth)

        # now calculate the pairwise distances
        for pair in dataset.keys():
            cat0, cat1 = pair[0], pair[1]
            dist = np.linalg.norm(world_coordinates[cat0] - world_coordinates[cat1])
            dataset[pair].append(dist)
        
        # TODO (@Thomas) probably no reasons to use pandas here
        df = pd.DataFrame(dataset)
        for col in df.columns.tolist():
            df[col] = df[col] / iqrs[col]
        
        pidx = np.indices((df.shape[1], df.shape[1])).reshape(2, -1)
        lcol = pd.MultiIndex.from_product([df.columns, df.columns],  names=[df.columns.name, df.columns.name])
        X = pd.DataFrame(df.values[:, pidx[0]] * df.values[:, pidx[1]],  columns=lcol)
        

        # load pca components
        newX = np.dot(X, components.T)

        weight = model.predict(newX)
        return weight

        

            
            
            



        
        
        
        

In [ ]:
image_ids = coco.getImgIds()
images = coco.loadImgs(image_ids)
stereo_frame_pairs = {}
for image in images:
    local_path = image['local_path']
    sfp_id = int(local_path.split('/')[-3])
    side = 'left' if 'left' in local_path else 'right'
    if sfp_id not in list(stereo_frame_pairs.keys()):
        stereo_frame_pairs[sfp_id] = {}
    stereo_frame_pairs[sfp_id][side] = image['id']
    
stereo_frame_pairs = list(stereo_frame_pairs.values())
    
    

In [ ]:
model = sm.load(MODEL_PATH)
components = np.load(COMPONENTS_PATH)
iqrs = pickle.load(open(IQRS_PATH, 'rb'))
weights = []
for i, stereo_frame_pair in enumerate(stereo_frame_pairs):
    left_image_id = stereo_frame_pair['left']
    left_image = coco.loadImgs([left_image_id])
    left_annotation_ids = coco.getAnnIds(imgIds=[left_image_id])
    left_annotations = coco.loadAnns(left_annotation_ids)
    
    right_image_id = stereo_frame_pair['right']
    right_image = coco.loadImgs([left_image_id])
    right_annotation_ids = coco.getAnnIds(imgIds=[right_image_id])
    right_annotations = coco.loadAnns(right_annotation_ids)
    categories = CATEGORIES
    filtered_categories = [kv for kv in categories if kv['id'] in MINIMAL_BODY_PART_LIST]
    
    left_annotations = match_body_parts(left_annotations, categories)
    right_annotations = match_body_parts(right_annotations, categories)
    if left_annotations and right_annotations:
        matched_annotations = left_right_matching(left_annotations, right_annotations, categories)
        for pair in matched_annotations:
            if pair[0] and pair[1]:
                weight = weight_estimator(pair[0], pair[1], filtered_categories, model, components, iqrs)
                if weight and 1000 < weight < 6000:
                    weights.append(weight)
    
    if i % 50 == 0:
        print(float(i) / len(stereo_frame_pairs))
            
    
    
    
        


In [ ]:
plt.hist(np.squeeze(weights))
plt.xlabel('Biomass (grams)')
plt.ylabel('Count')

In [ ]:
np.mean(np.squeeze(weights))